## LangChain FMP Data Tutorial
This notebook demonstrates how to use the langchain-fmp-data package to access financial market data through LangChain.

## Setup
First, let's install the required packages:



In [ ]:
!pip install -U langchain-fmp-data

### Credentials

To use this package, you'll need:

- An FMP (Financial Modeling Prep) API key from financialmodelingprep.com
- An OpenAI API key

Let's set up our environment variables:

In [2]:
import os

# Replace with your actual API keys
os.environ["FMP_API_KEY"] = "your-fmp-api-key"  # pragma: allowlist secret
os.environ["OPENAI_API_KEY"] = "your-openai-api-key"  # pragma: allowlist secret

It's also helpful (but not needed) to set up [LangSmith](https://smith.langchain.com/) for best-in-class observability:

In [3]:
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

## Using FMPDataToolkit
The FMPDataToolkit provides a collection of tools based on your natural language query. Here are some examples:


In [4]:
from langchain_fmp_data import FMPDataToolkit

# Example 1: Getting stock market data tools
market_toolkit = FMPDataToolkit(
    query="Get stock market prices and technical indicators",
    num_results=5
)
market_tools = market_toolkit.get_tools()
print("\nMarket Analysis Tools:")
for tool in market_tools:
    print(f"- {tool.name}: {tool.description}")

# Example 2: Getting fundamental analysis tools
fundamental_toolkit = FMPDataToolkit(
    query="Company financial statements and fundamental ratios",
    num_results=5
)
fundamental_tools = fundamental_toolkit.get_tools()
print("\nFundamental Analysis Tools:")
for tool in fundamental_tools:
    print(f"- {tool.name}: {tool.description}")

# Example 3: Getting economic data tools
economic_toolkit = FMPDataToolkit(
    query="Economic indicators and market statistics",
    num_results=5
)
economic_tools = economic_toolkit.get_tools()
print("\nEconomic Analysis Tools:")
for tool in economic_tools:
    print(f"- {tool.name}: {tool.description}")

## Integrating with LangChain Agents
You can use these tools with LangChain agents:


In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Initialize the LLM
llm = ChatOpenAI(temperature=0)

# Create toolkit and get tools
toolkit = FMPDataToolkit(
    query="Stock prices and company financials",
    num_results=3
)
tools = toolkit.get_tools()

# Create and run agent
agent = create_openai_functions_agent(llm, tools)
agent_executor = AgentExecutor(agent=agent, tools=tools)

# Example queries
queries = [
    "What's the current stock price of Apple?",
    "Show me Microsoft's revenue growth over the last year",
    "Compare Tesla and Ford's profit margins"
]

for query in queries:
    print(f"\nQuery: {query}")
    response = agent_executor.invoke({"input": query})
    print(f"Response: {response['output']}")

## Using FMPDataTool
The FMPDataTool provides a single, powerful tool that can answer financial queries by automatically selecting the right endpoints:


In [ ]:
from langchain_fmp_data import FMPDataTool
from langchain_fmp_data.tools import ResponseFormat

# Initialize the tool
tool = FMPDataTool()

# Example 1: Basic stock price query
query1 = "What's the current price of Bitcoin?"
response1 = tool.invoke({"query": query1})
print(f"\nQuery: {query1}")
print(f"Response: {response1}")

# Example 2: Complex financial analysis
query2 = "Compare the debt ratios of Apple and Microsoft"
response2 = tool.invoke({
    "query": query2,
    "response_format": ResponseFormat.BOTH  # Get both natural language and structured data
})
print(f"\nQuery: {query2}")
print(f"Response: {response2}")

# Example 3: Market analysis with technical indicators
query3 = "Show me the RSI and MACD indicators for Tesla stock"
response3 = tool.invoke({
    "query": query3,
    "response_format": ResponseFormat.DATA_STRUCTURE  # Get just the structured data
})
print(f"\nQuery: {query3}")
print(f"Response: {response3}")

## Advanced Usage
You can customize the tool's behavior:



In [16]:
# Initialize with custom settings
advanced_tool = FMPDataTool(
    max_iterations=50,  # Increase max iterations for complex queries
    temperature=0.2     # Adjust temperature for more/less focused responses
)

# Example of a complex multi-part analysis
query = """
Analyze Apple's financial health by:
1. Examining current ratios and debt levels
2. Comparing profit margins to industry average
3. Looking at cash flow trends
4. Assessing growth metrics
"""

response = advanced_tool.invoke({
    "query": query,
    "response_format": ResponseFormat.BOTH
})

print("Detailed Financial Analysis:")
print(response)